# NBA DX project

## Analyzer

In [1]:
import pandas as pd

from const import *
from utils import *

In [2]:
# import each JSON file in OUTPUT_DIR and assign it to a variable
# ex. 2010-11.json --> df_2010_11
for year in range(START_YEAR, END_YEAR):
    season = season_string(year)
    file = OUTPUT_DIR / f'{season}.json'
    df = pd.read_json(file)
    globals()[f'df_{year}'] = df
    print(f'Loaded {season} dataset')


Loaded 2009-10 dataset
Loaded 2010-11 dataset
Loaded 2011-12 dataset
Loaded 2012-13 dataset
Loaded 2013-14 dataset
Loaded 2014-15 dataset
Loaded 2015-16 dataset
Loaded 2016-17 dataset
Loaded 2017-18 dataset
Loaded 2018-19 dataset


In [4]:
df_2010

,https://www.basketball-reference.com/players/b/blackta01.html,https://www.basketball-reference.com/players/k/kellyry01.html,https://www.basketball-reference.com/players/b/battito01.html,https://www.basketball-reference.com/players/m/mahinia01.html,https://www.basketball-reference.com/players/d/derozde01.html,https://www.basketball-reference.com/players/h/hansbty01.html,https://www.basketball-reference.com/players/e/eyengch01.html,https://www.basketball-reference.com/players/w/willija04.html
url,https://www.basketball-reference.com/players/b...,https://www.basketball-reference.com/players/k...,https://www.basketball-reference.com/players/b...,https://www.basketball-reference.com/players/m...,https://www.basketball-reference.com/players/d...,https://www.basketball-reference.com/players/h...,https://www.basketball-reference.com/players/e...,https://www.basketball-reference.com/players/w...
season,2017-18,2014-15,2011-12,2017-18,2017-18,2014-15,2010-11,2010-11
name,Tarik Black,Ryan Kelly,Tony Battie,Ian Mahinmi,DeMar DeRozan,Tyler Hansbrough,Christian Eyenga,Jawad Williams
stats,"{'age': '26', 'team_id': 'HOU', 'lg_id': 'NBA'...","{'age': '23', 'team_id': 'LAL', 'lg_id': 'NBA'...","{'age': '35', 'team_id': 'PHI', 'lg_id': 'NBA'...","{'age': '31', 'team_id': 'WAS', 'lg_id': 'NBA'...","{'age': '28', 'team_id': 'TOR', 'lg_id': 'NBA'...","{'age': '29', 'team_id': 'TOR', 'lg_id': 'NBA'...","{'age': '21', 'team_id': 'CLE', 'lg_id': 'NBA'...","{'age': '27', 'team_id': 'CLE', 'lg_id': 'NBA'..."
seasons,"[2016-17, 2015-16, 2013-14, 2010-11, 2012-13, ...","[2016-17, 2015-16, 2013-14, 2010-11, 2012-13, ...","[2010-11, 2004-05, 1998-99, 1999-00, 1996-97, ...","[2016-17, 2013-14, 2015-16, 2018-19, 2010-11, ...","[2016-17, 2013-14, 2015-16, 2018-19, 2021-22, ...","[2013-14, 2015-16, 2010-11, 2012-13, 2007-08, ...","[2012-13, 2011-12, 2010-11]","[2010-11, 2004-05, 2001-02, 2002-03, 2009-10, ..."
games,[],[],[],[],[],[],[],[]
height,206,211,211,211,198,206,196,206
weight,113,104,104,118,99,113,95,98
birth_date,1991-11-22,1991-04-09,1976-02-11,1986-11-05,1989-08-07,1985-11-03,1989-06-22,1983-02-19
birth_place,"Memphis, Tennessee us","Carmel, New York us","Dallas, Texas us","Rouen, France fr","Compton, California us","Poplar Bluff, Missouri us","Kinshasa, Democratic Republic of the Congo cd","Cleveland, Ohio us"
